In [1]:
import numpy as np
import numpy.linalg as lin
from matplotlib import pyplot as plt

In [10]:
# Make grid space
def xgrid(L,n):
    dx=L/n
    grd = np.linspace(-L/2., L/2., num = n, endpoint=False) # Make grid
    grd = np.array(grd, dtype = np.float64)
    return grd

# Set frame for output
def plot_grid(L,n, grd):
    dx=L/n
    strline = ""
    if "float" in str(grd.dtype):
        tmp_grd = grd
    if "complex" in str(grd.dtype):
        tmp_grd = grd.real**2 + grd.imag**2
    for i in range(n):
        strline += "  %.5e " % tmp_grd[i]
    strline += "  %.6e  \n" % tmp_grd[0]
    return strline

# Make wave packet within grid space (also construct it's operator)
def wave(L, n, l, dt):
    grd=np.zeros(n, np.complex64)
    dx=np.float64(L/n)
    
    if pot_shape == 4:
        for i in range(n):                                   # wave packet initialize (gaussian) for bounded state
            if (i > (n*4)//10 and i < (n*6)//10):
                grd[i] = np.exp(-(i*dx-0.5*n*dx)**2/sigma)
            else:
                grd[i] = 0. + 0.j
        grd /= lin.norm(grd)                       # Fix to normalize
        for i in range(n):
            grd[i] = grd[i]*np.exp(1j*k*(i*dx-0.5*n*dx))  #Wave packet

    else:
        for i in range(n):                                   # wave packet initialize (gaussian)
            if (i > n*0/10 and i < n*4/10):
                grd[i] = np.exp(-(i*dx-0.3*n*dx)**2/sigma)
            else:
                grd[i] = 0. + 0.j
        grd /= lin.norm(grd)                       # Fix to normalize
        for i in range(n):
            grd[i] = grd[i]*np.exp(1j*k*(i*dx-0.3*n*dx))  #Wave packet
    
    return grd     

#Potential part
# Construct Potential Operator using Potential grid
def Potential(L,n):
    grd=np.zeros(n)
    grd[0] = 100000000
    grd[1] = 100000000
    grd[n-1]= 100000000
    if pot_shape == 0:                                   #no potential
        for i in range(1, n):
            grd[i] = 0                           # Make potential

    if pot_shape == 1:                                   #Step potential
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range((n//2),(n-2)):
            grd[i] = pot_height_eV              # eV unit
            grd[i] = grd[i]/27.211          # eV -> Har

    if pot_shape == 2:                                   #Potential barrier
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range((5*n)//10,(5*n)//10+barrier_thickness*10):
            grd[i] = pot_height_eV                    # eV unit
            grd[i] = grd[i]/27.211          # eV -> Har

    if pot_shape == 3:                                   #Double barrier
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range((45*n)//100-barrier_thickness*10,(45*n)//100):
            grd[i] = pot_height_eV                    # eV unit
            grd[i] = grd[i]/27.211          # eV -> Har
        for i in range((50*n)//100,(50*n)//100+barrier_thickness*10):
            grd[i] = pot_height_eV               # eV unit
            grd[i] = grd[i]/27.211          # eV -> Har

    if pot_shape == 4:                                   # Square well
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range(2,(n*4)//10):
            grd[i]= pot_height_eV
            grd[i] = grd[i]/27.211          # eV -> Har
        for i in range((n*6)//10,n-2):
            grd[i]= pot_height_eV
            grd[i] = grd[i]/27.211          # eV -> Har

    if pot_shape == 5:                              #Alpha
        for i in range(0, 500):
            grd[i]=0
        for i in range(500, 600):
            grd[i]= 66/27.211
        for i in range(600, n):
            grd[i]= 50/27.211

    if pot_shape == 6:                              #Triangle
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range(500, 601):
            grd[i] = pot_height_eV - 10*((i-500)/100)
            grd[i] = grd[i]/27.211          # eV -> Har

    return grd

# Define FDM points & coefficients (Here, 7-points FDM)
def fdmcoefficient(l):
    a=np.zeros((2*l+1,2*l+1))
    b=np.zeros(2*l+1)
    c=np.zeros(2*l+1)

    for i in range(0, 2*l+1):
        for j in range(0, 2*l+1):
            a[i,j]= (j-l)**i
    c[2]=2
    a = lin.inv(a)
    b= np.matmul(a, c)

    C=np.zeros((l+1))

    for i in range(0, l+1):
        C[i]=b[i+l]
    return C


###Laplacian and Hamiltonian
# Define Hamiltonian using Potential and Laplacian
def Laplacian(n, l, dx):
    C=fdmcoefficient(l)
    oprt=np.zeros((n,n))
    for i in range(n):
        for j in range(-l, l + 1, 1):
            k = i + j
            if (k >= n):
                k -= n
                oprt[i][k] = C[abs(j)] / (dx**2)
            elif (k<0):
                k += n
                oprt[i][k] = C[abs(j)] / (dx**2)
            else:
                oprt[i][k] = C[abs(j)] / (dx**2)
    return oprt


def Hamiltonian(L,n,l,dx):
    Hamiltonian = np.zeros((n,n))
    V = Potential(L, n)
    L = Laplacian(n, l, dx)                   # h- = 1, m_e = 1
    V_oprt=np.zeros((n,n))
    for i in range(0, n):
        V_oprt[i,i]=V[i]
    Hamiltonian = -L / 2. + V_oprt    # H = - (h-^2/2m) L + V
    return Hamiltonian

def time_operator(L,n,l,dt,dx):   

    if algorithm == 0:           # forward method
        oprt=np.zeros((n,n), np.complex64)
        H           = Hamiltonian(L, n, l, dx)
        exp_iHdt_0  = np.eye(n) - 1.j * H * dt
        oprt        = np.array(exp_iHdt_0, dtype = np.complex64)
        
    if algorithm == 1:           # backward method
        oprt=np.zeros((n,n), np.complex64)
        H           = Hamiltonian(L, n, l, dx)
        exp_iHdt_1  = np.eye(n) + 1.j * H * dt
        exp_iHdt_1  = np.array(exp_iHdt_1, dtype = np.complex64)
        oprt        = lin.inv(exp_iHdt_1)
        
    if algorithm == 2:           # Crank-Nickolson method
        oprt=np.zeros((n,n), np.complex64)
        H           = Hamiltonian(L, n, l, dx)
        exp_iHdt_2  = np.eye(n) + 1.j * H * dt / 2.
        exp_iHdt_2  = np.array(exp_iHdt_2, dtype = np.complex64)
        exp_iHdt_2  = lin.inv(exp_iHdt_2)
        exp_miHdt_2 = np.zeros_like(oprt)
        exp_miHdt_2 = np.eye(n) - 1.j * H * dt / 2.
        oprt        = np.dot(exp_iHdt_2, exp_miHdt_2)
    return oprt

def time_operatorback(L,n,l,dt,dx):   

    oprt=np.zeros((n,n), np.complex64)
    H           = Hamiltonian(L, n, l, dx)
    exp_iHdt_0  = np.eye(n) - 1.j * H * dt
    oprt        = np.array(exp_iHdt_0, dtype = np.complex64)
    oprt        = lin.inv(oprt)
        
    return oprt

In [11]:
# Input parameter
Lx = 100                # Box Length (Angstrom = 10^-10 m)
ngx = 1001              # Number of grid points (spacing = L/(n-1))
nstep = 10              # Number of time steps * 0.1fs
ewave = 13.6055         # Energy of packet (eV)
pot_shape = 0           # Shape of potential
                        # 0 = No potential
                        # 1 = Step potential
                        # 2 = Single wall
                        # 3 = Double wall
                        # 4 = Finite well (Packet starts at middle)
                        # 5 = Asymmetric potential barrier
                        # 6 = Triangular potential barrier (E-field)
pot_height_eV = 10      # eV
barrier_thickness = 10  # Thickness of the barrier(Angstrom = 10^-10 m)
                        # Only for Potential_shape = 2 or 3!

dispersion_gaussian = 2.646  # Spatial dispersion(sigma) of gaussian wave packet (Angstrom = 10^-10 m)

algorithm = 2

In [12]:
#Inputs & Unit conversion

k = (ewave*2/27.211)**0.5
sigma = dispersion_gaussian*1.88973
atomic_unit_of_time_to_fs = 2.418884326509E-2
angsL = Lx * 1.88973
l = 3
dt = 0.01/atomic_unit_of_time_to_fs
left = ngx//2
right = ngx//2

#Construict grid & wave packet
wave = wave(angsL, ngx, l, dt)           

# Save wave function as txt
f = open("wave.txt",'w')
xgrid=xgrid(angsL,ngx)

f.write("# t(fs) " + plot_grid(angsL,ngx,xgrid))
f.write("  0.0000 " + plot_grid(angsL, ngx,wave))
reflec = np.float64(0)
trans = np.float64(0)
poten = Potential(angsL, ngx)
inten=np.zeros(ngx)
saveprob = []
dx=angsL/ngx
next_step = time_operator(angsL,ngx,l,dt,dx)
back_step = time_operatorback(angsL,ngx,l,dt,dx)

#Time propagation of Wave packet using time operator
for i in range(nstep):
    for j in range(0, 10):
        if i < -1:
            wave = np.matmul(wave, back_step)
        else:
            wave=np.matmul(wave, next_step)
    reflec = 0
    trans = 0 
    #counting probability
    prob = np.abs(wave)**2
    for k in range(0, ngx):
        if k <= left :
            reflec += prob[k]
        if k > right :
            trans += prob[k]
    t = (i+1)*10 * dt * atomic_unit_of_time_to_fs
    f.write("%.6f" % t + plot_grid(angsL,ngx,wave))
    saveprob.append('%.6f    %.6f     %.6f\n' %(t,reflec,trans))
f.close()

# Print Potential Shape
f2 = open("Potential.txt",'w')
P=Potential(angsL,ngx)
f2.write(plot_grid(angsL,ngx,P))
f2.close()

# Print probability
f3 = open("Probablity.txt",'w')
f3.write("# t(fs)    Reflection  Transmission\n")
f3.writelines(saveprob)
f3.close()

### Visualization

In [13]:

st=0

L=[]
file=open('wave.txt', 'r')

while (1):
    line=file.readline()

    try:escape=line.index('\n')
    except:escape=len(line)

    if line:
        L.append(line[0:escape])
    else:
        break

file.close()

L2=[]
file=open('Potential.txt', 'r')

while (1):
    line=file.readline()

    try:escape=line.index('\n')
    except:escape=len(line)

    if line:
        L2.append(line[0:escape])
    else:
        break

file.close()

tmp=L2[0].split()
a=np.size(tmp)
pot=np.zeros(a)
for i in range(0, a):
    pot[i]=tmp[i]

x = np.linspace(st, Lx, ngx-1)
nL = np.size(L)
store = np.zeros((nL-1,ngx))
a = np.max(pot[5:np.size(pot)-5])

for i in range(1, nL):
    tmp=L[i].split()
    for j in range(0, ngx):
        store[i-1,j]=np.float64(tmp[j])

for i in range(0,nstep):
    plt.plot(x, store[i*1,1:ngx], label= 'Time = %.6f fs' %(store[i*1,0]))
    if a != 0 :
        plt.plot(x, pot[0:ngx-1]/a*0.15, label='Potential = %.3f eV ' %(a*27.211))
    plt.xlim(0,Lx)
    plt.ylim(0,0.2)
    plt.legend()
    plt.yticks([], [])
    plt.xlabel('Box [Angstrom]')
#    plt.show()
    plt.savefig('case_%03d.png' %i)
    plt.clf()
    
############# save animation ############
from PIL import Image
import glob
 
# Create the frames
frames = []
imgs = glob.glob("*.png")
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('movie.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

##### Erasing png files ##################
!del *.png

<Figure size 432x288 with 0 Axes>